# New Hire Training - Python Fundamentals: Day 2 Lesson Codes - Data Science Tools
This Jupyter Notebook file is a summary of codes demonstrated in class by The Marquee Group during the J.P. Morgan New Hire Training.

## Session 2 – Data Science Tools

This session will provide an overview of popular statistical packages in Python including NumPy, statsmodels and SciPy. During the session participants will gain experience in performing statistical analysis, linear regressions, time series forecasting and optimization. The session will also explore some more advanced functions in Pandas, including transforming time series data using the resample, aggregate and rolling functions.

### <font color = 'blue'> **Section 1 - Importing Packages and Data** </font>

In [ ]:
import pandas as pd
import numpy as np #log, matrix calculation, random numbers
import matplotlib.pyplot as plt

import statsmodels.api as sm #Linear regression
import statsmodels.tsa as tsa #time series analysis

In [ ]:
sp500 = pd.read_csv("ADAPT2021/StockData/SP500.csv",parse_dates=['Date'],index_col=['Date'])
aapl = pd.read_csv("ADAPT2021/StockData/AAPL.csv",parse_dates=['Date'],index_col=['Date'])

### <font color = 'blue'> **Section 2 - Linear Regression Model** </font>
- this example will calculate the Beta of Apple against S&P 500 using the CAPM model
- CAPM: return on equity = risk free rate + beta * market risk premium
- For simplicity, we will ignore in this example the risk free rate (assume it has been 0 for the period being analyzed) and perform a regression of AAPL's returns over S&P 500 returns
- The slope of the line of best fit will represent AAPL's beta
- Regression model will be run using OLS from statsmodels package

#### Creating Calculated Fields

In [ ]:
#%% Calculating Returns
sp500['Returns'] = sp500['Close'] / sp500['Close'].shift(1) - 1
    #daily return (Close/Prev Close) - 1

aapl['Returns'] = aapl['Close'].pct_change()

#### Merging Data Sets

In [ ]:
mergedData = sp500.merge(aapl, how='inner',
                         left_index=True, right_index=True,
                         suffixes=("_SP","_AAPL"))  
mergedData.dropna(inplace=True)
    #same as: mergedData = mergedData.dropna()

#### Regression Model
Steps:
- pick the model and the x/y variables
- fit the model
- look at the summary results and extract any important results (beta, r2, etc.)
- predict new data set (out of sample), or test old data (in-sample)
- plot predictions

In [ ]:
#%% Linear Regression Model
#run the model
#look at the summary results
#predict new data set (out of sample), can test old data (in-sample, backtesting)

#CAPM
# cost of equity = risk free rate + beta * MRP
    #MRP ---> market risk premium = excess returns of the market
    # S&P 500 as a proxy for the market
    
    #simplify assumption --> risk free rate = 0
    #return of AAPL = 0 + beta * MRP
        #MRP = S&P returns - 0

capm = sm.OLS(mergedData['Returns_AAPL'], mergedData['Returns_SP'])
         #  OLS(dependent variable, independ variable)

In [ ]:
results = capm.fit()
r2 = results.rsquared
results.summary() 
    #look at the r2
    #beta --> coefficient
    #look for p-stat to be less than 0.05

In [ ]:
results.params #just the coefficients or betas
# coefficients = results.params

In [ ]:
#Just the beta
beta = results.params[0]
beta

#### Predicting New Results

In [ ]:
#In-Sample forecast / "Backtesting"
mergedData['Predictions'] = results.predict(mergedData['Returns_SP'])

#Out-of sample forecast --> give your own data set to predict
mergedData

In [ ]:
#%% Visualize
plt.plot(mergedData['Returns_SP'],mergedData['Predictions'],'red')
plt.scatter(mergedData['Returns_SP'],mergedData['Returns_AAPL'],alpha=0.5)
plt.show()

#### Beta using Monthly Data


In [2]:
import pandas as pd
sp500 = pd.read_csv("ADAPT2021/StockData/SP500.csv",parse_dates=['Date'],index_col=['Date'])
aapl = pd.read_csv("ADAPT2021/StockData/AAPL.csv",parse_dates=['Date'],index_col=['Date'])

rules = {'Open':'first', 'Close':'last', 'Volume':'sum', 'High':'max', 'Low':'min'}
sp500_mo = sp500.resample('M').agg(rules)
aapl_mo = aapl.resample('M').agg(rules)

aapl_mo['Returns'] = aapl_mo['Close'].pct_change()
sp500_mo['Returns'] = sp500_mo['Close'].pct_change()

monthly_data = sp500_mo.merge(aapl_mo, how='outer',
                         left_index=True, right_index=True,
                         suffixes=("_SP","_AAPL")) 

monthly_data.dropna(inplace=True)
monthly_data

In [ ]:
monthly_data.plot.scatter(x='Returns_SP', y='Returns_AAPL')

In [ ]:
capm_mo = sm.OLS(monthly_data['Returns_AAPL'], monthly_data['Returns_SP']).fit()
monthly_data['Predictions'] = capm_mo.predict(monthly_data['Returns_SP'])
capm_mo.summary()

In [ ]:
monthly_data.plot.scatter(x='Returns_SP', y='Returns_AAPL')
plt.plot(monthly_data['Returns_SP'],monthly_data['Predictions'],'red')
plt.show()

### <font color = 'blue'> **Section 3 - Time Series Regression** </font>
- this example will run a regression using S&P 500 SMA

In [ ]:
sp500 = pd.read_csv("ADAPT2021/StockData/SP500.csv",parse_dates=['Date'],index_col=['Date'])


#Simple Moving Averages
sp500['sma'] = sp500['Close'].shift(1).rolling(window = 20, min_periods = 5).mean()

sp500['sma_1'] = sp500['Close'].shift(1)
sp500['sma_5'] = sp500['sma_1'].rolling(window = 5, min_periods = 5).mean()
sp500['sma_20'] = sp500['sma_1'].rolling(window = 20, min_periods = 20).mean()
sp500['sma_100'] = sp500['sma_1'].rolling(window = 100, min_periods = 100).mean()
sp500['sma_42'] = sp500['sma_1'].rolling(window = 42, min_periods = 42).mean()
sp500['sma_252'] = sp500['sma_1'].rolling(window = 252, min_periods = 252).mean()

#Exponential Moving Averages
sp500['ema_20'] = sp500['Close'].ewm(span=20).mean()

sp500.head(25)

In [ ]:
sp500[['Close','sma_42','sma_252']].plot()

In [ ]:
sp500.dropna(inplace=True)

# add constant to dataframe
sp500['alpha'] = 1
#regts = sm.OLS(sp500['Close'], sp500[['alpha','sma_1', 'sma_5', 'sma_20','sma_100','ema_20']]).fit()
#regts = sm.OLS(sp500['Close'], sp500[['sma_20', 'ema_20']]).fit()
regts = sm.OLS(sp500['Close'], sp500[['sma_42', 'sma_252']]).fit()
regts.summary()

In [ ]:
plt.plot(sp500.index, sp500['Close'])
# plt.plot(sp500.index, regts.predict(sp500[['sma_20', 'ema_20']]))
plt.plot(sp500.index, regts.predict(sp500[['sma_42', 'sma_252']]))
plt.legend(['Close','TSA'])
plt.show()

### <font color = 'blue'> **Section 4 - Optimization - SciPy** </font>

In [ ]:
#Import Optimization Packages
#from scipy.optimize import minimize, Bounds, LinearConstraint #Only for scipy version >1.0
from scipy.optimize import minimize

In [ ]:
#check version of scipy
import scipy
scipy.__version__

#### Optimization Function

In [ ]:
def optw(w, V):
    # Function returns the variance of the portfolio given weights and covar matrix
    return(np.matmul(np.matmul(w,V),w))

#### Preparing Data Set
Load all the stock data except for the S&P500 data.
- Resample to Monthly frequency.
- Calculate simple returns for each stock using the adjusted close.
- Combine the returns into a data frame where the index is date time and the columns are the individual stock returns.
- Drop any na rows.

In [ ]:
import os
stockdatalist = []
for file in os.listdir("ADAPT2021/StockData/"):
    if file.endswith(".csv"):
        stockdatalist.append(file)

# Remove SP500
stockdatalist.remove('SP500.csv')
#stockdatalist

In [ ]:
# Initalize an empty dataframe
rtns = pd.DataFrame()
ohlc_rule = {'Open':'first', 'High':'max',
                'Low':'min', 'Close':'last', 'Volume':'sum', 'Adj Close':'last'}

# Load the stock data, resample freq, calc simple returns, store in a df by ticker
for fileName in stockdatalist:
    ticker = fileName.replace(".csv","")
    filepath = "ADAPT2021/StockData/{}".format(fileName) 
    temp = pd.read_csv(filepath, index_col=0, parse_dates=True)
    temp = temp.resample('M').agg(ohlc_rule)
    temp = temp['Adj Close'].pct_change()
    rtns[ticker] = temp 

rtns.dropna(inplace=True)
rtns.head()

#Look at specific companies:
rtns = rtns[['PG','JNJ','IBM','AAPL','MCD','WMT','KO','NKE']]
#rtns = rtns.iloc[:,0:5].copy() #look at first 5 companies

#### Covariance Matrix and Avg Returns
- Calculate the covariance matrix of the stocks.
- Calculate the mean return of the stocks.
- Scale the mean and covariance matrix by 100 and 10,000 respectively.

In [ ]:
V = np.cov(rtns, rowvar=False) * 10000
mu = np.mean(rtns, axis=0) * 100
n = mu.shape[0]

print("# of tickers",n)
print(mu)
print(np.std(rtns)*100)

In [ ]:
#Correlation Matrix
# rtns.corr()
#Covariance Matrix
#rtns.cov() * 10000

#### Set up Constraints and Bounds
- Set initial values for the weights and define the expected return.
- Set up investment weights and returns constraints.
- Set up bounds for the weights (if no short selling allowed).

In [ ]:
# Set initial values for the weights and define the expected return
w = np.matrix([1/n] * n).T #initial guess equal weights
expect_return = 5

#lower/upper limits for the weights
#setting to 0<w<100% to restrict for short selling
bound = (0.0,1.0)
bounds = tuple(bound for asset in range(n))

#The following are Constraints using new Scipy (version >= 1.0)
# Weight constraint
#A = np.matrix([1]*n)
#constraint_1 = LinearConstraint(A, 1, 1)

# for the return constraint, the asset returns are the coefficients
#constraint_2 = LinearConstraint(mu, expect_return, np.inf)

#Constraints using older scipy version
    #x represents the variables in the opimization that are being sensitized
    #in this case they represent the weights in each security

constraint_1 = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
constraint_2 = {'type': 'ineq', 'fun': lambda x: np.sum(x * mu ) - expect_return}
    #eq --> means the fun has to be exactly equal to 0
    #ineq --> means the fun has to be positive
    #for constraint 2 using ineq to allow for returns to be > expect_return if possible

#### Run Optimization Model
- Set initial values for the weights and define the expected return.
- Set up investment weights and returns constraints.

In [ ]:
#Negative weights allowed (short selling)
#weight = minimize(optw, w, (V), constraints=(constraint_1,constraint_2), options={'maxiter':1000})

#No negative weights (no short selling) by adding boundaries
weight = minimize(optw, w, (V), constraints=(constraint_1,constraint_2),bounds=bounds,options={'maxiter':1000})


#weight
# weight.x.shape
weight.fun #calculated minimum variance at given expected return
weight.x #calculated optimal weights for each ticker
print("At an expected return of {:.2%} the minimum variance is {:.2f}".format(expect_return/100,weight.fun))

In [ ]:
weights = []
variances = []

for expect_return in np.linspace(0,5, 11):
    #constraint_2 = LinearConstraint(mu, expect_return, np.inf)
    constraint_2 = {'type': 'ineq', 'fun': lambda x: np.sum(x * mu ) - expect_return}
    
    #Run one of the next two lines (comment out the other)
        #Short-selling allowed:
    weight = minimize(optw, w, (V), constraints=(constraint_1,constraint_2), options={'maxiter':1000})
        #No short-selling:
    #weight = minimize(optw, w, (V), constraints=(constraint_1,constraint_2), bounds=bounds, options={'maxiter':1000})
    
    if weight.success == True:
        weights.append(weight.x)
        variances.append(weight.fun)
    else:
        print("no solution for return of ", expect_return)

In [ ]:
# Find the portfolio returns and variences
portmu = []
portvar = []

for w in weights:
    portmu.append(w @ mu)
    portvar.append(w @ V @ w)
    
portstd = np.sqrt(portvar)

In [ ]:
tickers = rtns.columns
results = pd.DataFrame(data=weights, columns=tickers)
results['Returns'] = portmu
results['Variances'] = portvar
results['STD'] = portstd

results

In [ ]:
#Plot of Portfolio Efficient Frontier
plt.scatter(rtns.std() * 100, rtns.mean() * 100)
plt.scatter(portstd, portmu)
plt.plot(portstd, portmu, linestyle='-.', color='black')
plt.title('Portfolio Frontier')
plt.xlim([0,10])
plt.ylim([0,5])
for x, y, ticker in zip(rtns.std()*100,rtns.mean()*100,rtns.mean().index):
    plt.text(x+0.2, y, ticker, ha='left')
plt.xlabel('Portfolio Risk (STD %)')
plt.ylabel('Portfolio Expected Return (%)')
plt.show()